In [21]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

In [37]:
%%sql
SELECT * FROM divtec_notasfiscais WHERE destino = '5501028-TANBOA COMERCIO DE FRUTA' ORDER BY destino;

,identificador,data,numeroNotaFiscal,destino,procedencia,produto,unidade,unidade_peso,quantidade,volume,nome_usuario_sistema,data_lancamento,informacao,machine_learning
0,23,2024-07-22,3175,5501028-TANBOA COMERCIO DE FRUTA,0143078-MONTENEGRO (RS),00101-ABACATE,KG,1.0,180.0,180.0,Andressa,NaN,NaN,NaN
1,24,2024-07-22,3175,5501028-TANBOA COMERCIO DE FRUTA,0143078-MONTENEGRO (RS),03201-TANGERINA COMUM / CAI,KG,1.0,540.0,540.0,Andressa,NaN,NaN,NaN
2,25,2024-07-22,3175,5501028-TANBOA COMERCIO DE FRUTA,0143078-MONTENEGRO (RS),03204-TANGERINA MONTENEGRINA,KG,1.0,720.0,720.0,Andressa,NaN,NaN,NaN
3,26,2024-07-22,3175,5501028-TANBOA COMERCIO DE FRUTA,0143078-MONTENEGRO (RS),01406-LARANJA SUCO,KG,1.0,360.0,360.0,Andressa,NaN,NaN,NaN
4,27,2024-07-22,3175,5501028-TANBOA COMERCIO DE FRUTA,0143078-MONTENEGRO (RS),01603-LIMAO TAITI,KG,1.0,180.0,180.0,Andressa,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557,286200,2024-10-01,5041,5501028-TANBOA COMERCIO DE FRUTA,0143078-MONTENEGRO (RS),01403-LARANJA LIMA (CEU),KG,1.0,432.0,432.0,Luiz,2024-10-04,NaN,NaN
558,286201,2024-10-01,5041,5501028-TANBOA COMERCIO DE FRUTA,0143078-MONTENEGRO (RS),01401-LARANJA BAIA (UMBIGO),KG,1.0,720.0,720.0,Luiz,2024-10-04,NaN,NaN
559,286202,2024-10-01,5041,5501028-TANBOA COMERCIO DE FRUTA,0143078-MONTENEGRO (RS),01406-LARANJA SUCO,KG,1.0,720.0,720.0,Luiz,2024-10-04,NaN,NaN
560,286203,2024-10-01,5041,5501028-TANBOA COMERCIO DE FRUTA,0143078-MONTENEGRO (RS),01603-LIMAO TAITI,KG,1.0,360.0,360.0,Luiz,2024-10-04,NaN,NaN


In [38]:
data = data.loc[data['produto'] == '00350-AMEIXA IMPORTADA']

In [39]:
df = data.copy(deep=True)

df_marcados = df[df['machine_learning'].notna()]
df_nao_marcados = df[df['machine_learning'].isna()]

le_destino = LabelEncoder()
le_produto = LabelEncoder()
le_procedencia = LabelEncoder()
le_unidade = LabelEncoder()

df_marcados['destino_encoded'] = le_destino.fit_transform(df_marcados['destino'])
df_marcados['produto_encoded'] = le_produto.fit_transform(df_marcados['produto'])
df_marcados['procedencia_encoded'] = le_procedencia.fit_transform(df_marcados['procedencia'])
df_marcados['unidade_encoded'] = le_unidade.fit_transform(df_marcados['unidade'])

model = IsolationForest(contamination=0.1)
model.fit(df_marcados[['destino_encoded', 'produto_encoded', 'procedencia_encoded', 'unidade_encoded','volume']])


IsolationForest(contamination=0.1)

In [41]:
df_nao_marcados['destino_encoded'] = le_destino.transform(df_nao_marcados['destino'])
df_nao_marcados['produto_encoded'] = le_produto.transform(df_nao_marcados['produto'])
df_nao_marcados['procedencia_encoded'] = le_procedencia.transform(df_nao_marcados['procedencia'])
df_nao_marcados['unidade_encoded'] = le_procedencia.transform(df_nao_marcados['unidade'])

df_nao_marcados['error'] = model.predict(df_nao_marcados[['destino_encoded', 'produto_encoded', 'procedencia_encoded', 'volume']])
df_nao_marcados['verificacao'] = df_nao_marcados['error'].apply(lambda x: 'NAO' if x == -1 else 'SIM')


ValueError: y contains previously unseen labels: 'PESO'

In [36]:
df_final = pd.concat([df_marcados, df_nao_marcados])
df_final[['nome_usuario_sistema', 'data', 'numeroNotaFiscal', 'procedencia', 'destino', 'produto', 'unidade', 'volume', 'machine_learning']]

,nome_usuario_sistema,data,numeroNotaFiscal,procedencia,destino,produto,unidade,volume,machine_learning
11,Andressa,2024-07-22,3175,0800000-(ARGENTINA),5501028-TANBOA COMERCIO DE FRUTA,00350-AMEIXA IMPORTADA,KG,27.0,SIM
60,Andressa,2024-07-29,3394,2200000-(ESPANHA),5501028-TANBOA COMERCIO DE FRUTA,00350-AMEIXA IMPORTADA,KG,36.0,SIM
104,Luiz,2024-08-01,3493,2200000-(ESPANHA),5501028-TANBOA COMERCIO DE FRUTA,00350-AMEIXA IMPORTADA,KG,18.0,SIM
133,Luiz,2024-08-05,3584,2200000-(ESPANHA),5501028-TANBOA COMERCIO DE FRUTA,00350-AMEIXA IMPORTADA,KG,54.0,SIM
179,Nicolly,2024-08-12,3771,2200000-(ESPANHA),5501028-TANBOA COMERCIO DE FRUTA,00350-AMEIXA IMPORTADA,KG,27.0,SIM
366,Simone,2024-09-03,4317,2200000-(ESPANHA),5501028-TANBOA COMERCIO DE FRUTA,00350-AMEIXA IMPORTADA,KG,27.0,SIM
555,Nicolly,2024-09-30,5015,2200000-(ESPANHA),5501028-TANBOA COMERCIO DE FRUTA,00350-AMEIXA IMPORTADA,PESO,36.0,NaN
